Updating previous work to pivot the new information obtained from the updated pct_sold_with efforts. I will NOT be looking at store level for this continued analysis.

In [1]:
import pandas as pd

In [2]:
#reading in necessary files for analysis, shoe recommendations

departmentindex = pd.read_csv('../data/departmentindex.csv')
storedata = pd.read_csv('../data/stores.csv')
products = pd.read_csv('../data/products.csv')
shoedivs = pd.read_csv('../newdata/topshoesdivision.csv')
shoeregions = pd.read_csv('../newdata/topshoesregion.csv')
shoestates = pd.read_csv('../newdata/topshoesstate.csv')
shoedistricts = pd.read_csv('../newdata/topshoesdistrict.csv')
fulldf = pd.read_csv('../data/fulldf.csv')

C:\Users\willi\AppData\Local\Temp\ipykernel_18748\3019035241.py:10: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  fulldf = pd.read_csv('../data/fulldf.csv')


In [3]:
#reading in necessary files for accessory recommendations

accdivs = pd.read_csv('../newdata/topaccdiv.csv')
accregions = pd.read_csv('../newdata/topaccregion.csv')
accstates = pd.read_csv('../newdata/topaccstate.csv')
accdistricts = pd.read_csv('../newdata/topaccdistrict.csv')

In [4]:
#Filtering shoe and accessory tables by count, where we only want to represent
#products sold together 3 or more times.

shoedivs = shoedivs[(shoedivs['count'] >=3 )]
shoeregions = shoeregions[(shoeregions['count'] >= 3)]
shoestates = shoestates[(shoestates['count'] >= 3)]
shoedistricts = shoedistricts[(shoedistricts['count'] >= 3)]


accdivs = accdivs[(accdivs['count'] >=3 )]
accregions = accregions[(accregions['count'] >= 3)]
accstates = accstates[(accstates['count'] >= 3)]
accdistricts = accdistricts[(accdistricts['count'] >= 3)]

In [5]:
# Concatenating all tables together to work in a future for loop.

allreclevels = pd.concat([shoedivs, shoeregions, shoedistricts, shoestates, accdivs, accregions, accdistricts, accstates], ignore_index=True, axis=0)

In [6]:
#re-organizing table and eliminating the count column.

allreclevels = allreclevels[['primary_stockno', 'secondary_stockno','secondaryshoe', 'pctsaleswith', 'count', 'level', 'hierarchyid']]

In [7]:
#only returning the necessary store information
districtdata = storedata[['DIV', 'STORE', 'REGION', 'DISTRICT', 'STORE_STATE_ALPHA']]

#creating a simplified dataframe for filtering with
districtdata = districtdata[['DISTRICT', 'STORE_STATE_ALPHA', 'REGION', 'DIV']]

#dropping duplicate rows.
districtdata = districtdata.drop_duplicates(keep='first')

In [8]:
#creating a list of districts and modifying column type to work with a loop
districtslist = districtdata['DISTRICT'].unique().astype(str).tolist()
districtdata['REGION'] = districtdata['REGION'].astype(str)

In [9]:
#creating an empty dataframe to push results to.
recommendations = pd.DataFrame(columns = ['district', 'primary_stockno', 'shoe1', 'level1', 'pct1', 'shoe2', 'level2', 'pct2', 'shoe3', 'level3', 'pct3'])

In [10]:
#subsetting to just shoe to shoe sales
allshoes = allreclevels[(allreclevels['secondaryshoe'] == 1)]
allshoes['hierarchyid'] = allshoes['hierarchyid'].astype(str)

#subsetting to just shoe to accessory sales
allacc = allreclevels[(allreclevels['secondaryshoe'] == 0)]
allacc['hierarchyid'] = allacc['hierarchyid'].astype(str)

C:\Users\willi\AppData\Local\Temp\ipykernel_18748\1368214614.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allshoes['hierarchyid'] = allshoes['hierarchyid'].astype(str)
C:\Users\willi\AppData\Local\Temp\ipykernel_18748\1368214614.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allacc['hierarchyid'] = allacc['hierarchyid'].astype(str)


In [11]:
#sanity checking the table was split appropriately and there isn't anything weird going on.

len(allacc) + len(allshoes) == len(allreclevels)

True

If everything above works, this should be returning the most accurate version of product recommendations by pct_sold_with.

In [12]:
#this is a program that starts at the district level and tries to fill in recommendations for every product offered
#slowly working it's way up the hierarchy chain until as many products as possible are filled in at the lowest possible hierarchy level.
#district/product combos that can not be populated are pushed to an errors list. Will investigate, but I suspect those are products not bought in the listed district.
#current formatting of this cell returns shoe recommendations for a shoe purchase.
productrecs1 = []
productlist = allreclevels.primary_stockno.unique().tolist()
errors = []

for district in districtslist:
    
    districtfiltered = allshoes[(allshoes['hierarchyid'] == district) & (allshoes['level'] == 'district')]
    
    for product in productlist:
        
        temp = districtfiltered[(districtfiltered['primary_stockno'] == product)]
        temp2 = allshoes[(allshoes['hierarchyid'].isin(districtdata[(districtdata['DISTRICT'] == int(district))].STORE_STATE_ALPHA.tolist())) & (allshoes['primary_stockno'] == product)]
        temp3 = allshoes[(allshoes['hierarchyid'].isin(districtdata[(districtdata['DISTRICT'] == int(district))].REGION.astype(str).unique().tolist())) & (allshoes['primary_stockno'] == product) & (allshoes['level'] == 'region')]
        temp4 = allshoes[(allshoes['level'] == 'div') & (allshoes['hierarchyid'] == '8') & (allshoes['primary_stockno'] == product)]
        
        if len(temp) == 3: #111
            productrecs1.append([district, temp.iloc[0,0], 
                                 temp.iloc[0, 1], temp.iloc[0, -2], temp.iloc[0, 4],
                                 temp.iloc[1, 1], temp.iloc[1, -2], temp.iloc[1, 4],
                                 temp.iloc[2, 1], temp.iloc[2, -2], temp.iloc[2, 4]])
            
        elif len(temp) == 2:           
            if len(temp) == 2 and len(temp2) >= 1: #112
                productrecs1.append([district, temp.iloc[0,0], 
                                     temp.iloc[0, 1], temp.iloc[0, -2], temp.iloc[0, 4],
                                     temp.iloc[1, 1], temp.iloc[1, -2], temp.iloc[1, 4],
                                     temp2.iloc[0, 1], temp2.iloc[0, -2], temp2.iloc[0, 4]])
            elif len(temp) == 2 and len(temp3) >= 1: #113
                productrecs1.append([district, temp.iloc[0,0], 
                                     temp.iloc[0, 1], temp.iloc[0, -2], temp.iloc[0, 4],
                                     temp.iloc[1, 1], temp.iloc[1, -2], temp.iloc[1, 4],
                                     temp3.iloc[0, 1], temp3.iloc[0, -2], temp3.iloc[0, 4]])
            elif len(temp) == 2 and len(temp4) >= 1: #114
                productrecs1.append([district, temp.iloc[0,0], 
                                     temp.iloc[0, 1], temp.iloc[0, -2], temp.iloc[0, 4],
                                     temp.iloc[1, 1], temp.iloc[1, -2], temp.iloc[1, 4],
                                     temp4.iloc[0, 1], temp4.iloc[0, -2], temp4.iloc[0, 4]])
            
        elif len(temp) == 1: 
            if len(temp2) >= 2: #122
                productrecs1.append([district, temp.iloc[0,0], 
                                     temp.iloc[0, 1], temp.iloc[0, -2], temp.iloc[0, 4],
                                     temp2.iloc[0, 1], temp2.iloc[0, -2], temp2.iloc[0, 4],
                                     temp2.iloc[1, 1], temp2.iloc[1, -2], temp2.iloc[1, 4]])
            elif len(temp2) == 1 and len(temp3)  >= 1: #123
                productrecs1.append([district, temp.iloc[0,0], 
                                     temp.iloc[0, 1], temp.iloc[0, -2], temp.iloc[0, 4],
                                     temp2.iloc[0, 1], temp2.iloc[0, -2], temp2.iloc[0, 4],
                                     temp3.iloc[0, 1], temp3.iloc[0, -2], temp3.iloc[0, 4]])
            elif len(temp2) == 1 and len(temp4) >= 1: #124
                productrecs1.append([district, temp.iloc[0,0], 
                                     temp.iloc[0, 1], temp.iloc[0, -2], temp.iloc[0, 4],
                                     temp2.iloc[0, 1], temp2.iloc[0, -2], temp2.iloc[0, 4],
                                     temp4.iloc[0, 1], temp4.iloc[0, -2], temp4.iloc[0, 4]])
            elif len(temp) == 1 and len(temp3) >= 2: #133
                productrecs1.append([district, temp.iloc[0,0], 
                                     temp.iloc[0, 1], temp.iloc[0, -2], temp.iloc[0, 4],
                                     temp3.iloc[0, 1], temp3.iloc[0, -2], temp3.iloc[0, 4],
                                     temp3.iloc[1, 1], temp3.iloc[1, -2], temp3.iloc[1, 4]])
            elif len(temp) == 1 and len(temp3) >= 1 and len(temp4) >=1: #134
                productrecs1.append([district, temp.iloc[0,0], 
                                     temp.iloc[0, 1], temp.iloc[0, -2], temp.iloc[0, 4],
                                     temp3.iloc[0, 1], temp3.iloc[0, -2], temp3.iloc[0, 4],
                                     temp4.iloc[0, 1], temp4.iloc[0, -2], temp4.iloc[0, 4]])
            elif len(temp) == 1 and len(temp4) >= 2: #144
                productrecs1.append([district, temp.iloc[0,0], 
                                     temp.iloc[0, 1], temp.iloc[0, -2], temp.iloc[0, 4],
                                     temp4.iloc[0, 1], temp4.iloc[0, -2], temp4.iloc[0, 4],
                                     temp4.iloc[1, 1], temp4.iloc[1, -2], temp4.iloc[1, 4]])
        
        elif len(temp2) == 3: #222
            productrecs1.append([district, temp2.iloc[0,0], 
                                temp2.iloc[0, 1], temp2.iloc[0, -2], temp2.iloc[0, 4],
                                temp2.iloc[1, 1], temp2.iloc[1, -2], temp2.iloc[1, 4],
                                temp2.iloc[2, 1], temp2.iloc[2, -2], temp2.iloc[2, 4]])
        
        elif len(temp2) == 2: 
            if len(temp2) == 2 and len(temp3) >= 1: #223
                productrecs1.append([district, temp2.iloc[0,0], 
                                     temp2.iloc[0, 1], temp2.iloc[0, -2], temp2.iloc[0, 4],
                                     temp2.iloc[1, 1], temp2.iloc[1, -2], temp2.iloc[1, 4],
                                     temp3.iloc[0, 1], temp3.iloc[0, -2], temp3.iloc[0, 4]])
            elif len(temp2) == 2 and len(temp4) >= 1: #224
                productrecs1.append([district, temp2.iloc[0,0], 
                                     temp2.iloc[0, 1], temp2.iloc[0, -2], temp2.iloc[0, 4],
                                     temp2.iloc[1, 1], temp2.iloc[1, -2], temp2.iloc[1, 4],
                                     temp4.iloc[0, 1], temp4.iloc[0, -2], temp4.iloc[0, 4]])
            elif len(temp2) == 1 and len(temp3) >= 2: #233
                productrecs1.append([district, temp2.iloc[0,0], 
                                     temp2.iloc[0, 1], temp2.iloc[0, -2], temp2.iloc[0, 4],
                                     temp3.iloc[0, 1], temp3.iloc[0, -2], temp3.iloc[0, 4],
                                     temp3.iloc[1, 1], temp3.iloc[1, -2], temp3.iloc[1, 4]])
            elif len(temp2) == 1 and len(temp3) == 1 and len(temp4) >= 1: #234
                productrecs1.append([district, temp2.iloc[0,0], 
                                     temp2.iloc[0, 1], temp2.iloc[0, -2], temp2.iloc[0, 4],
                                     temp3.iloc[0, 1], temp3.iloc[0, -2], temp3.iloc[0, 4],
                                     temp4.iloc[0, 1], temp4.iloc[0, -2], temp4.iloc[0, 4]])
            elif len(temp2) == 1 and len(temp4) >= 2: #244
                productrecs1.append([district, temp2.iloc[0,0], 
                                     temp2.iloc[0, 1], temp2.iloc[0, -2], temp2.iloc[0, 4],
                                     temp4.iloc[0, 1], temp4.iloc[0, -2], temp4.iloc[0, 4],
                                     temp4.iloc[1, 1], temp4.iloc[1, -2], temp4.iloc[1, 4]])
        
        elif len(temp) == 0 and len(temp3) == 3: #333
            productrecs1.append([district, temp3.iloc[0,0], 
                                temp3.iloc[0, 1], temp3.iloc[0, -2], temp3.iloc[0, 4],
                                temp3.iloc[1, 1], temp3.iloc[1, -2], temp3.iloc[1, 4],
                                temp3.iloc[2, 1], temp3.iloc[2, -2], temp3.iloc[2, 4]])
        
        elif len(temp3) == 2 and len(temp4) >= 1: #334
            productrecs1.append([district, temp3.iloc[0,0], 
                                temp3.iloc[0, 1], temp3.iloc[0, -2], temp3.iloc[0, 4],
                                temp3.iloc[1, 1], temp3.iloc[1, -2], temp3.iloc[1, 4],
                                temp4.iloc[0, 1], temp4.iloc[0, -2], temp4.iloc[0, 4]])
        
        elif len(temp3) == 1 and len(temp4) >= 2: #344
            productrecs1.append([district, temp3.iloc[0,0], 
                                temp3.iloc[0,1], temp3.iloc[0, -2], temp3.iloc[0, 4],
                                temp4.iloc[0,1], temp4.iloc[0, -2], temp4.iloc[0, 4],
                                temp4.iloc[1,1], temp4.iloc[1, -2], temp4.iloc[1, 4]])
        elif len(temp4) == 3: #444
            productrecs1.append([district, temp4.iloc[0,0], 
                                temp4.iloc[0, 1], temp4.iloc[0, -2], temp4.iloc[0, 4],
                                temp4.iloc[1, 1], temp4.iloc[1, -2], temp4.iloc[1, 4],
                                temp4.iloc[2, 1], temp4.iloc[2, -2], temp4.iloc[2, 4]])
        
        else:
            errors.append([district, product])
                  

In [13]:
shoerecs = pd.DataFrame(productrecs1)

In [14]:
shoerecs.columns = ['district', 'primary_stockno', 'shoe1', 'level1', 'count1', 'shoe2', 'level2', 'count2', 'shoe3', 'level3', 'count3']

In [15]:
shoerecs.to_csv('../newdata/shoerecscount.csv', index=False)

In [16]:
pd.DataFrame(errors).to_csv('../newdata/shoeerrors.csv', index=False)

In [17]:
accrecs = pd.read_csv('../newdata/accrecscount.csv')
shoerecs = pd.read_csv('../newdata/shoerecscount.csv')

In [18]:
shoerecs['level1'].value_counts()

div                  32149
region               15711
store_state_alpha    12469
district              7340
Name: level1, dtype: int64

In [19]:
shoerecs['level2'].value_counts()

div                  34702
store_state_alpha    14224
region               13663
district              5080
Name: level2, dtype: int64

In [20]:
shoerecs['level3'].value_counts()

div                  37047
region               14180
store_state_alpha    12584
district              3858
Name: level3, dtype: int64

In [21]:
#finding what percentage of recommendations come from a given hierarchy for shoes.

print((32149+34702+37047) / (len(shoerecs)*3) * 100)
print((15711+13633+14180) / (len(shoerecs)*3) * 100)
print((12469+14224+12584) / (len(shoerecs)*3) * 100)
print((3858+5080+7340) / (len(shoerecs)*3) * 100)

51.179515977281575
21.43965479022891
19.34760870314817
8.018442713798047


In [22]:
accrecs['level1'].value_counts()

div                  31558
region                8820
store_state_alpha     5375
district              4818
Name: level1, dtype: int64

In [23]:
accrecs['level2'].value_counts()

div                  32349
region                8464
store_state_alpha     5920
district              3838
Name: level2, dtype: int64

In [24]:
accrecs['level3'].value_counts()

div                  33406
region                8570
store_state_alpha     5533
district              3062
Name: level3, dtype: int64

In [25]:
#finding what percentage of recommendations come from a given hierarchy for accessories.

print((31558+32349+33406) / (len(accrecs)*3) * 100)
print((8820+8464+8570) / (len(accrecs)*3) * 100)
print((5375+5920+5533) / (len(accrecs)*3) * 100)
print((3062+3838+4818) / (len(accrecs)*3) * 100)

64.14282230263721
17.041387356390025
11.091996071529795
7.723794269442961


In [26]:
noproductsdistrict = pd.DataFrame(fulldf.groupby('district').primary_stockno.nunique()).reset_index()

In [27]:
accrecsdistrict = pd.DataFrame(accrecs.groupby('district').primary_stockno.nunique()).reset_index()

In [28]:
shoerecsdistrict = pd.DataFrame(shoerecs.groupby('district').primary_stockno.nunique()).reset_index()

In [29]:
accrecsdistrict.columns = ['district', 'noofaccrecs']
shoerecsdistrict.columns = ['district', 'noofshoerecs']

In [30]:
noproductsdistrict.columns = ['district', 'totalproducts']

In [31]:
noproductsdistrict['district'] = noproductsdistrict['district'].astype(float).astype(int)

In [32]:
shoerecsdistrict['district'] = shoerecsdistrict['district'].astype(str)
accrecsdistrict['district'] = accrecsdistrict['district'].astype(str)
noproductsdistrict['district'] = noproductsdistrict['district'].astype(str)

In [33]:
recstats = shoerecsdistrict.merge(accrecsdistrict, on='district')

In [34]:
recstats = recstats.merge(noproductsdistrict, on='district')

In [35]:
recstats['pctwithaccrecs'] = recstats['noofaccrecs'] / recstats['totalproducts'] * 100
recstats['pctwithshoerecs'] = recstats['noofshoerecs'] / recstats['totalproducts'] * 100

In [36]:
recstats.to_csv('../newdata/recommendationstats.csv', index=False)

In [37]:
recstats.pctwithaccrecs.describe()

count    67.000000
mean     50.322931
std       4.630794
min      40.992750
25%      47.240983
50%      49.377049
75%      51.996947
max      68.059424
Name: pctwithaccrecs, dtype: float64

In [38]:
recstats.pctwithshoerecs.describe()

count    67.000000
mean     67.350877
std       6.375152
min      56.274400
25%      63.671396
50%      66.509751
75%      69.700896
max      93.686165
Name: pctwithshoerecs, dtype: float64

In [39]:
recstats.reset_index(drop=True)

,district,noofshoerecs,noofaccrecs,totalproducts,pctwithaccrecs,pctwithshoerecs
0,1,1009,736,1496,49.197861,67.446524
1,2,993,758,1637,46.304215,60.659743
2,3,995,740,1429,51.784465,69.629111
3,4,1038,827,1489,55.540631,69.711216
4,5,991,730,1580,46.202532,62.721519
...,...,...,...,...,...,...
62,69,992,756,1373,55.061908,72.250546
63,70,1009,733,1077,68.059424,93.686165
64,71,992,746,1364,54.692082,72.727273
65,72,1002,750,1437,52.192067,69.728601
